In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras import Model

In [ ]:
# Load the COCO dataset
data_dir = 'coco_dataset'

# Get the list of image files
image_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.jpg')]

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(image_files, [0] * len(image_files), test_size=0.1, random_state=42)

print(f"Number of training images: {len(X_train)}")
print(f"Number of testing images: {len(X_test)}")

In [ ]:
# Plot some sample images
plt.figure(figsize=(12, 6))
for i in range(4):
    plt.subplot(1, 4, i+1)
    plt.imshow(plt.imread(X_train[i]))
    plt.axis('off')

plt.show()

In [ ]:
# Perform image augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    contrast_stretch_range=(0.8, 1.2)
)

# Fit and generate augmented training data
train_generator = train_datagen.flow_from_directory(
    directory=data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
# Get the updated number of training and testing images
X_train, y_train = next(train_generator)
print(f"Number of training images after augmentation: {len(X_train)}")
print(f"Number of testing images: {len(X_test)}")

In [ ]:
# Normalize the training data
X_train = X_train / 255.0
# Build a convolutional neural network
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(80, activation='softmax'))

model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))X_test = np.array([plt.imread(f) for f in X_test]) / 255.0

In [ ]:
# Show the training and testing accuracy
print(f"Training accuracy: {history.history['accuracy'][-1]:.2f}")
print(f"Testing accuracy: {history.history['val_accuracy'][-1]:.2f}")

In [ ]:
# Build a Faster R-CNN
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in model.layers:
    layer.trainable = False

x = model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(80, activation='softmax')(x)

model = Model(inputs=model.input, outputs=predictions)
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

In [ ]:
# Show the training and testing accuracy
print(f"Training accuracy: {history.history['accuracy'][-1]:.2f}")
print(f"Testing accuracy: {history.history['val_accuracy'][-1]:.2f}")